Model time.

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

os.chdir('/Users/wynnephilpott/Documents/missing pauldron/Data Science/datasets')

In [2]:
df = pd.read_csv('dfs.csv')

In [3]:
df.columns

Index(['iid', 'gender', 'order', 'partner', 'pid', 'match', 'int_corr',
       'samerace', 'age_o', 'race_o', 'pf_o_att', 'pf_o_sin', 'pf_o_int',
       'pf_o_fun', 'pf_o_amb', 'pf_o_sha', 'dec_o', 'attr_o', 'sinc_o',
       'intel_o', 'fun_o', 'amb_o', 'like_o', 'prob_o', 'met_o', 'dec', 'attr',
       'sinc', 'intel', 'fun', 'amb', 'like', 'prob', 'met'],
      dtype='object')

The 'o's stand for parter's ratings so get rid of them (except for race, age) for a model that is to predict what the actual person(not the parter) will do. Daters were given 4-5 minutes with each partner. After they spoke, the daters then rated their parters on 'attr','sinc', 'intel', 'fun', 'amb'. Additionally, they put down how much they like the person (score from 1 to 10) and finally how probable they thought that thier partner liked them.

In [14]:
X = df[['int_corr','attr','samerace', 'age_o', 'race_o','sinc', 'intel', 'fun', 'amb', 'like', 'prob', 'met']]

y = df['dec']

Let's make our training and test sets for our machine learning algorithms. We'll start with Logistic Regression.

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [17]:
from sklearn.linear_model import LogisticRegression

lgm = LogisticRegression()
lgm.fit(X_train,y_train)

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [18]:
pred = lgm.predict(X_test)

In [19]:
from sklearn.metrics import classification_report as cr
from sklearn.metrics import confusion_matrix as cm

In [21]:
print(cr(y_test,pred))
print(cm(y_test,pred))

              precision    recall  f1-score   support

           0       0.79      0.80      0.79      1077
           1       0.72      0.70      0.71       781

   micro avg       0.76      0.76      0.76      1858
   macro avg       0.75      0.75      0.75      1858
weighted avg       0.76      0.76      0.76      1858

[[864 213]
 [235 546]]


I like to use F1 score weighted average as it considers both precision and recall. This will be how I measure model success. Let's try to bump up that .76!

In [24]:
#I want to see a summary of the coefficients of this model

import statsmodels.formula.api as sm

model = sm.Logit(y_train, X_train)
result = model.fit()

Optimization terminated successfully.
         Current function value: 0.496559
         Iterations 6


In [25]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    dec   No. Observations:                 4335
Model:                          Logit   Df Residuals:                     4323
Method:                           MLE   Df Model:                           11
Date:                Sun, 19 May 2019   Pseudo R-squ.:                  0.2759
Time:                        15:52:02   Log-Likelihood:                -2152.6
converged:                       True   LL-Null:                       -2972.7
                                        LLR p-value:                     0.000
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
int_corr      -0.0381      0.124     -0.306      0.759      -0.282       0.206
attr           0.3767      0.028     13.421      0.000       0.322       0.432
samerace      -0.3114      0.079     -3.947      0.000      -0.466      -0.157
age_o         -0.1325      0.008    -17.167      0.000      -0.148      -0.117
race_o        -0.1894      0.030     -6.238      0.000      -0.249      -0.130
sinc          -0.2276      0.033     -6.943      0.000      -0.292      -0.163
intel         -0.1225      0.039     -3.103      0.002      -0.200      -0.045
fun            0.0980      0.031      3.119      0.002       0.036       0.160
amb           -0.1682      0.030     -5.550      0.000      -0.228      -0.109
like           0.5994      0.039     15.558      0.000       0.524       0.675
prob           0.1595      0.022      7.358      0.000       0.117       0.202
met           -0.0712      0.038     -1.863      0.062      -0.146       0.004
==============================================================================
"""

**Females**

Based on our findings from the EDA, it makes sense that we should apply one model to Females and a different one to males.

In [26]:
female = df[df['gender']==0]

In [28]:
X = female[['int_corr','attr','samerace', 'age_o', 'race_o','sinc', 'intel', 'fun', 'amb', 'like', 'prob', 'met']]
y = female['dec']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)


lgm.fit(X_train,y_train)
pred = lgm.predict(X_test)

print(cr(y_test,pred))
print(cm(y_test,pred))

              precision    recall  f1-score   support

           0       0.77      0.83      0.80       582
           1       0.68      0.58      0.63       348

   micro avg       0.74      0.74      0.74       930
   macro avg       0.72      0.71      0.71       930
weighted avg       0.73      0.74      0.74       930

[[485  97]
 [145 203]]


/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


:(

**Males**

In [17]:
male = df[df['gender']==1]

In [18]:
X = male[['int_corr','attr','samerace', 'age_o', 'race_o','sinc', 'intel', 'fun', 'amb', 'like', 'prob', 'met']]
y = male['dec']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)


lgm.fit(X_train,y_train)
pred = lgm.predict(X_test)

print(cr(y_test,pred))
print(cm(y_test,pred))

              precision    recall  f1-score   support

           0       0.82      0.78      0.80       481
           1       0.77      0.82      0.80       448

   micro avg       0.80      0.80      0.80       929
   macro avg       0.80      0.80      0.80       929
weighted avg       0.80      0.80      0.80       929

[[373 108]
 [ 81 367]]


/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Not too bad actually.

Lets do partitioning beyond male/female and later attempt to engineer features.